In [44]:
from astropy.io import fits
from matplotlib import pyplot
import scipy.optimize as opt
import numpy as np
import pathlib
import re
import os
%matplotlib inline

In [64]:
p = pathlib.Path("/scratch/datasets/astro_deconv_2019/")

output = "/scratch/datasets/astro_deconv_2019_cleanbeam"

In [14]:
psf_file = list(p.glob("*-psf.fits"))[0]

In [15]:
data = fits.open(psf_file)[0].data.squeeze()

In [17]:
def twoD_Gaussian(pos, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = pos
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

In [67]:
def fit(data):
    s = data.shape[0]
    
    x = np.linspace(0, s-1, s)
    y = np.linspace(0, s-1, s)
    x, y = np.meshgrid(x, y)
    initial_guess = (1, s/2, s/2, 20, 20, 0, 0 )
    popt, pcov = opt.curve_fit(twoD_Gaussian, (x, y), data.flatten(), p0=initial_guess,  maxfev=10000)
    return twoD_Gaussian((x, y), *popt).reshape(s, s)

In [68]:
for psf_file in list(p.glob("*-wsclean-psf.fits")):

    index = re.match(r'(\d+)-wsclean-psf\.fits', pathlib.Path(psf_file).name)[1]
    target = f'{output}/{index}-clean-beam.fits'
    if pathlib.Path(target).is_file():
        #print('skipping ' + target)
        continue
    try:
        print(psf_file)
        f = fits.open(psf_file)
        data = f[0].data.squeeze()
        clean_beam = fit(data)
        hdu = fits.PrimaryHDU(clean_beam)
        hdul = fits.HDUList([hdu])
        hdul.header = f[0].header
        hdul.writeto(target, overwrite=True)
    except Exception as e:
        print (f"error: {e}")
        pass

/scratch/datasets/astro_deconv_2019/609-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8271-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5224-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1674-wsclean-psf.fits
error: Optimal parameters not found: Number of calls to function has reached maxfev = 10000.
/scratch/datasets/astro_deconv_2019/8646-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9473-wsclean-psf.fits
error: Optimal parameters not found: Number of calls to function has reached maxfev = 10000.
/scratch/datasets/astro_deconv_2019/4158-wsclean-psf.fits
error: Optimal parameters not found: Number of calls to function has reached maxfev = 10000.
/scratch/datasets/astro_deconv_2019/7470-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/380-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6798-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9850-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7348-wsclean-psf.fits
/scratch/datasets/astro_dec